In [1]:
import numpy as np
import os

# Context

Explain here HMM etc ...

# Sequential Montecarlo

## Linear Gaussian State Space Model

In [2]:
# 4.1. Linear Gaussian State Space Model
# Imports
from ipmcmc.smc import smc
from ipmcmc.linear_gaussian_state_model import *
from scipy.spatial.transform import Rotation as R
np.random.seed(420)

# Parameters
r = R.from_rotvec(np.array([7*np.pi/10, 3*np.pi/10, np.pi/20]))
rotation_matrix = r.as_dcm()
scaling_matrix = 0.99*np.eye(3)
beta = np.random.dirichlet(np.ones(20)*0.2, 3).transpose()
alpha = scaling_matrix@rotation_matrix
t_max = 50
mu = np.array([0, 1, 1])
start_var = 0.1*np.eye(3)
omega = np.eye(3)
sigma = 0.1*np.eye(20)

n_particles = 50

# If we want to change the parameters
assert np.all(np.linalg.eigvals(start_var) > 0)
assert np.all(np.linalg.eigvals(omega) > 0)
assert np.all(np.linalg.eigvals(sigma) > 0)

linear_transition_model = [LinearTransition(default_mean=np.zeros(3),default_cov=omega,default_alpha=alpha) for t in range(0, t_max)]
linear_proposals = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearProposal(default_mean=np.zeros(3),default_cov=omega,default_alpha=alpha) for t in range(1, t_max)]
linear_observation_model = [LinearObservation(default_mean=np.zeros(20),default_cov=sigma,default_beta=beta) for t in range(0, t_max)]

linear_states, linear_observations = linear_gaussian_state_space(t_max=t_max, mu=mu, start_var=start_var, transition_var=omega, noise_var=sigma, transition_coeffs=alpha, observation_coeffs=beta)

linear_particles, linear_weights, linear_ancestors = smc(observations=linear_observations, n_particles=n_particles,
       transition_model=linear_transition_model, proposals=linear_proposals, observation_model=linear_observation_model)

## Nonlinear State Space Model

In [0]:
# 4.2. Nonlinear State Space Model
# Imports
from ipmcmc.smc import smc
from ipmcmc.non_linear_gaussian_state_model import *
import numpy as np

# Parameters
np.random.seed(420)
t_max = 50
mu = 0
start_std = np.sqrt(5)
omega = np.sqrt(10)
sigma = np.sqrt(10)

n_particles = 50

nonlinear_transition_model = [NonLinearTransition(default_mean=0,default_std=omega) for t in range(0, t_max)]
nonlinear_proposals = [NonLinearMu(default_mean=mu, default_std=start_std)]+[NonLinearProposal(default_mean=0,default_std=omega) for t in range(1, t_max)]
nonlinear_observation_model = [NonLinearObservation(default_mean=0,default_std=sigma) for t in range(0, t_max)]

nonlinear_states, nonlinear_observations = nonlinear_gaussian_state_space(t_max=t_max, mu=mu, start_std=start_std, transition_std=omega, noise_std=sigma)

nonlinear_particles, nonlinear_weights, nonlinear_ancestors = smc(observations=nonlinear_observations, n_particles=n_particles,
       transition_model=nonlinear_transition_model, proposals=nonlinear_proposals, observation_model=nonlinear_observation_model)

In [0]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(9, 4), sharey=True)
ax1.hist(nonlinear_states, bins=100)
ax2.hist(nonlinear_observations, bins=100)
plt.show()

RESULT ANALYSIS

In [0]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def plot_ancestors(ancestors: np.ndarray, img: str):
    raise DeprecationWarning("TODO: Refactor according to the new shape (time, n_particles)")
    # TODO: Refactor according to the new shape (time, n_particles)
    G = nx.Graph()

    for p_i in range(ancestors.shape[0]):
        G.add_node("p={};t={}".format(p_i, 0))
    
    for t_i in range(ancestors.shape[1]):
        for p_i, a_t_i in enumerate(ancestors[:,t_i]):
            G.add_node("p={};t={}".format(int(p_i), int(t_i+1)))
            G.add_edge("p={};t={}".format(int(a_t_i), int(t_i)) ,"p={};t={}".format(int(p_i), int(t_i+1)))
    
    coeff = 10000
    fixed_positions = {node: (int(node.split(';')[0][2:])*coeff,int(node.split(';')[1][2:])*coeff)
        for node in G.nodes
    }

    plt.figure(figsize=(120,120))
    pos = nx.spring_layout(G,pos=fixed_positions, fixed=G.nodes)
    nx.draw_networkx(G,pos)
    plt.savefig(img)
    plt.close()
   
    return G